In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
# Load the dataset
data = pd.read_csv("11.csv")

# Separate data into different dataframes based on data types
spo2_data = data[data['data_type'] == 'spo2'].copy()
heart_rate_data = data[data['data_type'] == 'hr'].copy()
steps_data = data[data['data_type'] == 'steps'].copy()
hrv_data = data[data['data_type'] == 'hrv'].copy()
br_data = data[data['data_type'] == 'br'].copy()
sleep_data = data[data['data_type'] == 'sleep'].copy()

# Convert timestamps to datetime format for each dataframe
spo2_data['time'] = pd.to_datetime(spo2_data['time'])
heart_rate_data['time'] = pd.to_datetime(heart_rate_data['time'])
steps_data['time'] = pd.to_datetime(steps_data['time'])
hrv_data['time'] = pd.to_datetime(hrv_data['time'])
br_data['time'] = pd.to_datetime(br_data['time'])
sleep_data['time'] = pd.to_datetime(sleep_data['time'])

# Sort dataframes based on time and their respective formats
spo2_data.sort_values(by='time', inplace=True)
heart_rate_data.sort_values(by='time', inplace=True)
steps_data.sort_values(by='time', inplace=True)
hrv_data.sort_values(by='time', inplace=True)
br_data.sort_values(by='time', inplace=True)
sleep_data.sort_values(by='time', inplace=True)

In [3]:
heart_rate_data=heart_rate_data[['value', 'time']]

In [4]:
heart_rate_data

,value,time
1,80,2023-12-04 00:00:00+00:00
3,82,2023-12-04 00:01:00+00:00
4,82,2023-12-04 00:02:00+00:00
5,83,2023-12-04 00:03:00+00:00
6,79,2023-12-04 00:04:00+00:00
...,...,...
2539,87,2023-12-23 13:00:00+00:00
2540,85,2023-12-23 13:01:00+00:00
2541,85,2023-12-23 13:02:00+00:00
2542,86,2023-12-23 13:03:00+00:00


In [5]:
steps_data=steps_data[['value', 'time']]

In [6]:
steps_data

,value,time
60,9,2023-12-04 00:58:00+00:00
63,9,2023-12-04 00:59:00+00:00
65,13,2023-12-04 01:00:00+00:00
67,7,2023-12-04 01:01:00+00:00
69,22,2023-12-04 01:02:00+00:00
...,...,...
2518,8,2023-12-22 15:41:00+00:00
2522,12,2023-12-22 15:44:00+00:00
2527,18,2023-12-22 15:47:00+00:00
2528,6,2023-12-22 15:48:00+00:00


In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load use and non-use data
use_data = pd.read_csv('ID11_Use.csv')
non_use_data = pd.read_csv('ID11_None.csv')

# Filter rows with 'Melon' in the substance_fruit_label column
use_data = use_data[use_data['substance_fruit_label'] == 'Nectarine']

# Standardize the timestamp format in 'hawaii_use_time'
use_data['hawaii_use_time'] = use_data['hawaii_use_time'].apply(lambda x: x.split('.')[0] if '.' in x else x)
use_data['hawaii_use_time'] = pd.to_datetime(use_data['hawaii_use_time'], format='%Y-%m-%d %H:%M:%S').dt.tz_localize(None)

# Convert timestamps to ensure they are timezone-naive for non-use data
non_use_data['hawaii_createdat_time'] = pd.to_datetime(non_use_data['hawaii_createdat_time'], format='%Y-%m-%d %H:%M:%S').dt.tz_localize(None)

# Ensure heart rate and steps data use the same timezone-naive datetime index
heart_rate_data['time'] = pd.to_datetime(heart_rate_data['time']).dt.tz_localize(None)
steps_data['time'] = pd.to_datetime(steps_data['time']).dt.tz_localize(None)

heart_rate_data.set_index('time', inplace=True)
steps_data.set_index('time', inplace=True)

# Convert the 'value' column to numeric, coercing errors
heart_rate_data['value'] = pd.to_numeric(heart_rate_data['value'], errors='coerce')
steps_data['value'] = pd.to_numeric(steps_data['value'], errors='coerce')

# Remove duplicate timestamps
heart_rate_data = heart_rate_data[~heart_rate_data.index.duplicated(keep='first')]
steps_data = steps_data[~steps_data.index.duplicated(keep='first')]

# Reindex and forward fill to handle missing data
heart_rate_data = heart_rate_data.reindex(pd.date_range(start=heart_rate_data.index.min(), end=heart_rate_data.index.max(), freq='min'), method='ffill')
steps_data = steps_data.reindex(pd.date_range(start=steps_data.index.min(), end=steps_data.index.max(), freq='min'), method='ffill').fillna(0)

# Initialize the StandardScaler
scaler = StandardScaler()

def process_label_data(label_data, heart_rate_data, steps_data, scaler, time_column):
    results = []
    for _, row in label_data.iterrows():
        timestamp = row[time_column]
        lower_bound = timestamp - pd.Timedelta(hours=1)
        upper_bound = timestamp + pd.Timedelta(hours=1)

        # Extract data within the window
        hr_window = heart_rate_data.loc[lower_bound:upper_bound]
        steps_window = steps_data.loc[lower_bound:upper_bound]

        # Resample and calculate mean every 4 minutes to get 30 points
        hr_means = hr_window.resample('4min').mean().iloc[:30]  # Ensure 30 data points
        steps_means = steps_window.resample('4min').mean().iloc[:30]

        if len(hr_means) == 30 and len(steps_means) == 30:
            # Standardize the means
            hr_scaled = scaler.fit_transform(hr_means.values.reshape(-1, 1))
            steps_scaled = scaler.fit_transform(steps_means.values.reshape(-1, 1))

            results.append({
                'timestamp': timestamp,
                'heart_rate_means': hr_scaled.flatten().tolist(),
                'steps_means': steps_scaled.flatten().tolist()
            })

    return pd.DataFrame(results)

# Process use and non-use labels
use_features = process_label_data(use_data, heart_rate_data, steps_data, scaler, 'hawaii_use_time')
non_use_features = process_label_data(non_use_data, heart_rate_data, steps_data, scaler, 'hawaii_createdat_time')

# Output results
print("Use Features Data:")
print(use_features.head())
print("Non-Use Features Data:")
print(non_use_features.head())


Use Features Data:
            timestamp                                   heart_rate_means  \
0 2023-12-04 13:23:55  [-0.2351159491584021, 2.1804588709621635, -1.1...   
1 2023-12-05 01:15:02  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2 2023-12-05 21:01:18  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3 2023-12-06 08:20:31  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4 2023-12-08 13:05:46  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                         steps_means  
0  [-1.284156719038521, -2.555416598717975, -2.17...  
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
Non-Use Features Data:
            timestamp                                   heart_rate_means  \
0 2023-12-09 09:37:20  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1 2023-12-10 11:02:

In [8]:
# Print the shape of the use features data
print("Shape of Use Features Data:", use_features.shape)

# Print the shape of the non-use features data
print("Shape of Non-Use Features Data:", non_use_features.shape)

Shape of Use Features Data: (20, 3)
Shape of Non-Use Features Data: (5, 3)


In [9]:
import pandas as pd

# Adding label columns
use_features['state'] = 'use'
use_features['state_val'] = 1

non_use_features['state'] = 'non_crave'
non_use_features['state_val'] = 0

# Combining the dataframes
combined_data = pd.concat([use_features, non_use_features])

# Converting timestamps to datetime and sorting
combined_data['timestamp'] = pd.to_datetime(combined_data['timestamp'])
combined_data_sorted = combined_data.sort_values(by='timestamp').reset_index(drop=True)

# This combined_data_sorted is now ready for use in your neural network model
print(combined_data_sorted)


             timestamp                                   heart_rate_means  \
0  2023-12-04 13:23:55  [-0.2351159491584021, 2.1804588709621635, -1.1...   
1  2023-12-05 01:15:02  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  2023-12-05 21:01:18  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  2023-12-06 08:20:31  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  2023-12-08 13:05:46  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
5  2023-12-08 16:49:22  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
6  2023-12-08 21:29:21  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
7  2023-12-09 09:37:20  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
8  2023-12-10 02:41:28  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
9  2023-12-10 11:02:38  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
10 2023-12-10 11:02:50  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
11 2023-12-11 00:29:08  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

In [10]:
import tensorflow as tf

# Load pretrained models
encoder_hr = tf.keras.models.load_model('heart_rate_encoder')
encoder_steps = tf.keras.models.load_model('steps_encoder')

OSError: No file or directory found at heart_rate_encoder

In [ ]:
import numpy as np

# Function to generate embeddings using the loaded models
def generate_embeddings(hr_data, steps_data, encoder_hr, encoder_steps):
    # Convert lists to numpy arrays and ensure they are in the correct shape
    hr_data = np.array(hr_data.tolist())
    steps_data = np.array(steps_data.tolist())

    # Reshape data if required by the model, assuming the model expects shape (samples, features)
    if hr_data.ndim == 1:
        hr_data = hr_data.reshape(-1, 1)
    if steps_data.ndim == 1:
        steps_data = steps_data.reshape(-1, 1)

    # Generate embeddings
    hr_embeddings = encoder_hr.predict(hr_data)
    steps_embeddings = encoder_steps.predict(steps_data)

    return hr_embeddings, steps_embeddings

# Use the function with your data
hr_embeddings, steps_embeddings = generate_embeddings(
    combined_data_sorted['heart_rate_means'],
    combined_data_sorted['steps_means'],
    encoder_hr,
    encoder_steps
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
from sklearn.metrics import recall_score, precision_score, confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight

# Combine the heart rate and steps embeddings
combined_embeddings = np.concatenate([hr_embeddings, steps_embeddings], axis=1)

# Labels from your existing DataFrame
labels = combined_data_sorted['state_val'].values

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_embeddings, labels, test_size=0.2, random_state=42)

# Define the model with regularization and batch normalization
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(16, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall()])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.1,
    class_weight=class_weight_dict,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate thresholds on training set
probabilities_train = model.predict(X_train)
thresholds = np.arange(0, 1.01, 0.01)

# Store training results
training_results = []

print("\nTraining Set Evaluation:")
for threshold in thresholds:
    predicted_classes = (probabilities_train > threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_train, predicted_classes).ravel()
    recall = tp / (tp + fn)  # Sensitivity
    specificity = tn / (tn + fp)  # Specificity
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    training_results.append((threshold, recall, specificity, accuracy))
    print(f"Threshold: {threshold:.2f}, Sensitivity: {recall:.2f}, Specificity: {specificity:.2f}, Accuracy: {accuracy:.2f}")

# Convert training results to DataFrame for easy viewing
training_results_df = pd.DataFrame(training_results, columns=['Threshold', 'Sensitivity', 'Specificity', 'Accuracy'])

# Select best threshold based on a specific criterion
# Here, we select the threshold with the highest sensitivity while keeping specificity above 0.5
best_threshold_row = training_results_df[(training_results_df['Sensitivity'] > 0.9) & (training_results_df['Specificity'] > 0.5)]
if not best_threshold_row.empty:
    best_threshold = best_threshold_row.iloc[0]['Threshold']
else:
    best_threshold = 0.5  # Default threshold if criteria is not met

print(f"\nBest Threshold from Training Set: {best_threshold}")

# Evaluate the best threshold on the test set
probabilities_test = model.predict(X_test)
predicted_classes_test = (probabilities_test > best_threshold).astype(int)
tn, fp, fn, tp = confusion_matrix(y_test, predicted_classes_test).ravel()
recall_test = tp / (tp + fn)  # Sensitivity
specificity_test = tn / (tn + fp)  # Specificity
accuracy_test = (tp + tn) / (tp + tn + fp + fn)

print(f"\nTest Set Evaluation at Best Threshold ({best_threshold}):")
print(f"Sensitivity: {recall_test:.2f}")
print(f"Specificity: {specificity_test:.2f}")
print(f"Accuracy: {accuracy_test:.2f}")

# Store test results
test_results = []

for threshold in thresholds:
    predicted_classes = (probabilities_test > threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, predicted_classes).ravel()
    recall = tp / (tp + fn)  # Sensitivity
    specificity = tn / (tn + fp)  # Specificity
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    test_results.append((threshold, recall, specificity, accuracy))
    print(f"Threshold: {threshold:.2f}, Sensitivity: {recall:.2f}, Specificity: {specificity:.2f}, Accuracy: {accuracy:.2f}")

# Convert test results to DataFrame for easy viewing
test_results_df = pd.DataFrame(test_results, columns=['Threshold', 'Sensitivity', 'Specificity', 'Accuracy'])

# Find the threshold where specificity is around 0.9 and sensitivity is around 0.5
desired_specificity = 0.9
desired_sensitivity_range = (0.4, 0.6)  # Considering sensitivity around 0.5

filtered_results = test_results_df[
    (test_results_df['Specificity'] >= desired_specificity) & 
    (test_results_df['Sensitivity'] >= desired_sensitivity_range[0]) & 
    (test_results_df['Sensitivity'] <= desired_sensitivity_range[1])
]

# Print the filtered results
print("\nFiltered Thresholds with Specificity >= 0.9 and Sensitivity ~ 0.5:")
print(filtered_results)

# Plot the results for better visualization
plt.figure(figsize=(12, 6))
plt.plot(test_results_df['Threshold'], test_results_df['Sensitivity'], label='Sensitivity')
plt.plot(test_results_df['Threshold'], test_results_df['Specificity'], label='Specificity')
plt.plot(test_results_df['Threshold'], test_results_df['Accuracy'], label='Accuracy')
plt.axhline(y=0.9, color='r', linestyle='--', label='Desired Specificity (0.9)')
plt.axhline(y=0.5, color='g', linestyle='--', label='Desired Sensitivity (0.5)')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Sensitivity, Specificity, and Accuracy vs. Threshold')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
Training
Threshold: 0.45, Sensitivity: 0.90, Specificity: 0.57, Accuracy: 0.80
Threshold: 0.46, Sensitivity: 0.87, Specificity: 0.83, Accuracy: 0.85
Threshold: 0.47, Sensitivity: 0.75, Specificity: 0.87, Accuracy: 0.79
Threshold: 0.48, Sensitivity: 0.73, Specificity: 0.96, Accuracy: 0.80
Threshold: 0.49, Sensitivity: 0.69, Specificity: 0.96, Accuracy: 0.77
Threshold: 0.50, Sensitivity: 0.67, Specificity: 0.96, Accuracy: 0.76
Threshold: 0.51, Sensitivity: 0.65, Specificity: 0.96, Accuracy: 0.75
Threshold: 0.52, Sensitivity: 0.62, Specificity: 0.96, Accuracy: 0.72
Threshold: 0.53, Sensitivity: 0.62, Specificity: 0.96, Accuracy: 0.72
Threshold: 0.54, Sensitivity: 0.56, Specificity: 1.00, Accuracy: 0.69
Threshold: 0.55, Sensitivity: 0.48, Specificity: 1.00, Accuracy: 0.64
Threshold: 0.56, Sensitivity: 0.44, Specificity: 1.00, Accuracy: 0.61
Test:
Threshold: 0.45, Sensitivity: 0.75, Specificity: 0.33, Accuracy: 0.68
Threshold: 0.46, Sensitivity: 0.75, Specificity: 0.33, Accuracy: 0.68
Threshold: 0.47, Sensitivity: 0.62, Specificity: 0.33, Accuracy: 0.58
Threshold: 0.48, Sensitivity: 0.62, Specificity: 0.67, Accuracy: 0.63
Threshold: 0.49, Sensitivity: 0.56, Specificity: 0.67, Accuracy: 0.58
Threshold: 0.50, Sensitivity: 0.56, Specificity: 0.67, Accuracy: 0.58
Threshold: 0.51, Sensitivity: 0.56, Specificity: 0.67, Accuracy: 0.58
Threshold: 0.52, Sensitivity: 0.44, Specificity: 0.67, Accuracy: 0.47
Threshold: 0.53, Sensitivity: 0.38, Specificity: 0.67, Accuracy: 0.42
Threshold: 0.54, Sensitivity: 0.38, Specificity: 0.67, Accuracy: 0.42
Threshold: 0.55, Sensitivity: 0.25, Specificity: 0.67, Accuracy: 0.32
Threshold: 0.56, Sensitivity: 0.19, Specificity: 0.67, Accuracy: 0.26